# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint



### Not for Grading

## NLP with CNNs

In [ ]:
#@title Case Study Walkthrough
#@markdown  NLP with CNNs
from IPython.display import HTML

HTML("""<video width="320" height="240" controls>
  <source src="https://cdn.talentsprint.com/talentsprint/archives/sc/aiml/aiml_2018_b7_hyd/preview_videos/nlp_with_cnns.mp4">
</video>
""")

The objective of this experiment is to see the application of Convolutional Neural Networks in NLP.

####Note that this case study based on this [paper.](http://www.aclweb.org/anthology/D14-1181)

### Setup Steps

In [ ]:
#@title Please enter your registration id to start:  { run: "auto", display-mode: "form" }
Id="2100121"#@param{type:"string"}

In [ ]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password="5142192291"#@param{type:"string"}

In [ ]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()
  
notebook= "U3W15_CS_NLP_with_CNNs" #name of the notebook
Answer = "Ungraded"
def setup():
    ipython.magic("sx pip3 install gensim")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/week9/Exp2/AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.bin&sa=D&source=hangouts&ust=1550651743825000&usg=AFQjCNHh2LSwNi9czsqAAuBLvx_vDeUE_Q")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "feedback_experiments_input" : Comments, "notebook" : notebook}

      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:        
        print(r["err"])
        return None   
      else:
        print("Your submission is successful.")
        print("Ref Id:",submission_id)
        print("Date of submission: ",r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://aiml.iiith.talentsprint.com/notebook_submissions")
        # print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
      return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if not Additional: 
      raise NameError
    else:
      return Additional  
  except NameError:
    print ("Please answer Additional Question")
    return None
def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError 
    else: 
      return Answer
  except NameError:
    print ("Please answer Question")
    return None

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
    
else:
  print ("Please complete Id and Password cells before running setup")


Setup completed successfully


In [ ]:
!ls

AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.bin    train_5500.label
AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.bin.1  U3W15_CS_NLP_with_CNNs.ipynb
sample_data


##Importing required packages

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter, OrderedDict
import nltk
import re
from copy import deepcopy
flatten = lambda l: [item for sublist in l for item in sublist]
random.seed(1024)

## Code for accessing CUDA

In [ ]:
USE_CUDA = torch.cuda.is_available()
gpus = [0]
torch.cuda.set_device(gpus[0])
FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor

## Function to split the data in to batches

In [ ]:
def getBatch(batch_size, train_data):
    random.shuffle(train_data)
    sindex = 0
    eindex = batch_size
    while eindex < len(train_data):
        batch = train_data[sindex: eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch

## Function to add the padding to batches if required

In [ ]:
def pad_to_batch(batch):
    x,y = zip(*batch)
    max_x = max([s.size(1) for s in x])
    x_p = []
    for i in range(len(batch)):
        if x[i].size(1) < max_x:
            x_p.append(torch.cat([x[i], Variable(LongTensor([word2index['<PAD>']] * (max_x - x[i].size(1)))).view(1, -1)], 1))
        else:
            x_p.append(x[i])
    return torch.cat(x_p), torch.cat(y).view(-1)

## Function to prepare the sequence

In [ ]:
def prepare_sequence(seq, to_index):
    idxs = list(map(lambda w: to_index[w] if to_index.get(w) is not None else to_index["<UNK>"], seq))
    #print(idxs)
    return Variable(LongTensor(idxs))

## Data load & Preprocessing

### TREC question dataset(http://cogcomp.org/Data/QA/QC/)

The following command gets the required TREC question dataset.

In [ ]:
!wget http://cogcomp.org/Data/QA/QC/train_5500.label

--2020-12-19 08:07:35--  http://cogcomp.org/Data/QA/QC/train_5500.label
Resolving cogcomp.org (cogcomp.org)... 192.17.90.133
Connecting to cogcomp.org (cogcomp.org)|192.17.90.133|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cogcomp.seas.upenn.edu/Data/QA/QC/train_5500.label [following]
--2020-12-19 08:07:35--  https://cogcomp.seas.upenn.edu/Data/QA/QC/train_5500.label
Resolving cogcomp.seas.upenn.edu (cogcomp.seas.upenn.edu)... 158.130.57.77
Connecting to cogcomp.seas.upenn.edu (cogcomp.seas.upenn.edu)|158.130.57.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 335858 (328K)
Saving to: ‘train_5500.label.1’

train_5500.label.1  100%[===================>] 327.99K  --.-KB/s    in 0.04s   

2020-12-19 08:07:35 (8.92 MB/s) - ‘train_5500.label.1’ saved [335858/335858]



Task involves
classifying a question into 6 question
types (whether the question is about person,
location, numeric information, etc.)

## Load the data

In [ ]:
data = open('train_5500.label', 'r', encoding='latin-1').readlines()

In [ ]:
data[:5]

['DESC:manner How did serfdom develop in and then leave Russia ?\n',
 'ENTY:cremat What films featured the character Popeye Doyle ?\n',
 "DESC:manner How can I find a list of celebrities ' real names ?\n",
 'ENTY:animal What fowl grabs the spotlight after the Chinese Year of the Monkey ?\n',
 'ABBR:exp What is the full form of .com ?\n']

## Split the data by seperating the labels

In [ ]:
data = [[d.split(':')[1][:-1], d.split(':')[0]] for d in data]

In [ ]:
data[:5]

[['manner How did serfdom develop in and then leave Russia ?', 'DESC'],
 ['cremat What films featured the character Popeye Doyle ?', 'ENTY'],
 ["manner How can I find a list of celebrities ' real names ?", 'DESC'],
 ['animal What fowl grabs the spotlight after the Chinese Year of the Monkey ?',
  'ENTY'],
 ['exp What is the full form of .com ?', 'ABBR']]

In [ ]:
X, y = list(zip(*data))
X = list(X)

In [ ]:
print(X[:5])
print(y[:5])

['manner How did serfdom develop in and then leave Russia ?', 'cremat What films featured the character Popeye Doyle ?', "manner How can I find a list of celebrities ' real names ?", 'animal What fowl grabs the spotlight after the Chinese Year of the Monkey ?', 'exp What is the full form of .com ?']
('DESC', 'ENTY', 'DESC', 'ENTY', 'ABBR')


## Print the labels in the data

In [ ]:
set(y)

{'ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM'}

## Number masking 

In [ ]:
for i, x in enumerate(X):
    X[i] = re.sub('\d', '#', x).split()

Replacing the numbers with # (hash)

It reduces the search space. 

For example, 

my birthday is 12.22 ==> my birthday is ##.##

In [ ]:
X[:2]

[['manner',
  'How',
  'did',
  'serfdom',
  'develop',
  'in',
  'and',
  'then',
  'leave',
  'Russia',
  '?'],
 ['cremat',
  'What',
  'films',
  'featured',
  'the',
  'character',
  'Popeye',
  'Doyle',
  '?']]

## Building the Vocabulary

In [ ]:
vocab = list(set(flatten(X)))
print(len(vocab))
print(vocab)

9117
['lawnmower', 'Avery', 'states', 'Edmonton', 'Loco-Motion', 'windshield', 'bomb', 'swimmer', 'roles', 'viewing', 'quantity', 'S.O.S.', 'online', 'Motors', 'February', 'operate', 'transport', 'five', 'initial', 'lump', 'basidiomycetes', 'Spangled', 'province', 'Organization', 'appearances', 'tail', 'Merrie', 'Barbara', 'Jazz', 'stripe', 'Desire', 'Sinclair', 'Lane', 'amezaiku', 'agreement', 'Montenegro', 'Belle', 'protestors', 'seaport', 'fire', 'Mutiny', 'Pibb', 'tabulates', 'falls', 'god', 'decorations', 'Gaulle', 'salesman', 'net', 'S.', 'vitreous', 'Fordham', 'before', 'G.', 'Arts', 'Butch', 'Gross', 'Tours', 'shoulder', 'poet', 'Around', 'kill', 'Madre', 'Mcbeal', 'distinction', 'Harry', 'spices', 'Booth', 'pizza', 'Twain', "'m", 'Yom', 'flier', 'chiropodist', 'Abbey', 'Muhammad', 'standards', 'Cromwell', 'Be', 'since', 'Lagoon', 'hydroelectric', 'walls', 'May', 'aircraft', 'Get', 'liners', 'rounded', 'port', 'fastest-growing', 'capita', 'feminist', 'stings', 'July', 'offered'

## Check for number of classes

In [ ]:
len(set(y)) # num of class

6

## Create the index to words in the vocabulary

In [ ]:
word2index={'<PAD>': 0, '<UNK>': 1}
print(len(word2index))

2


In [ ]:
print(word2index.get('<PAD>'))
print(word2index.get('<UNK>'))
print(word2index.get(vocab[1]))

0
1
None


In [ ]:
for vo in vocab:
    if word2index.get(vo) is None:
        word2index[vo] = len(word2index)
#print(word2index)
index2word = {v:k for k, v in word2index.items()}
#print(index2word)

## Create the index to target

In [ ]:
target2index = {}

for cl in set(y):
    if target2index.get(cl) is None:
        target2index[cl] = len(target2index)

index2target = {v:k for k, v in target2index.items()}

## Preparing the data in tensor format

In [ ]:
X_p, y_p = [], []
for pair in zip(X,y):
    ## Create the indexes for the list of split words of questions present in X and changing to tensor format
    X_p.append(prepare_sequence(pair[0], word2index).view(1, -1)) 
    ## Changes the format of labels to tensor format
    y_p.append(Variable(LongTensor([target2index[pair[1]]])).view(1, -1))

## Zipping both the data and labels and shuffle randomly

In [ ]:
data_p = list(zip(X_p, y_p))
random.shuffle(data_p)

## Split the data into train and test

In [ ]:
train_data = data_p[: int(len(data_p) * 0.9)]
test_data = data_p[int(len(data_p) * 0.9):]

## Load Pretrained word vector

In [ ]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.bin', binary=True)
len(model.index2word)

3000000

In [ ]:
model.index2word

In [ ]:
word2index.keys()

dict_keys(['<PAD>', '<UNK>', 'lawnmower', 'Avery', 'states', 'Edmonton', 'Loco-Motion', 'windshield', 'bomb', 'swimmer', 'roles', 'viewing', 'quantity', 'S.O.S.', 'online', 'Motors', 'February', 'operate', 'transport', 'five', 'initial', 'lump', 'basidiomycetes', 'Spangled', 'province', 'Organization', 'appearances', 'tail', 'Merrie', 'Barbara', 'Jazz', 'stripe', 'Desire', 'Sinclair', 'Lane', 'amezaiku', 'agreement', 'Montenegro', 'Belle', 'protestors', 'seaport', 'fire', 'Mutiny', 'Pibb', 'tabulates', 'falls', 'god', 'decorations', 'Gaulle', 'salesman', 'net', 'S.', 'vitreous', 'Fordham', 'before', 'G.', 'Arts', 'Butch', 'Gross', 'Tours', 'shoulder', 'poet', 'Around', 'kill', 'Madre', 'Mcbeal', 'distinction', 'Harry', 'spices', 'Booth', 'pizza', 'Twain', "'m", 'Yom', 'flier', 'chiropodist', 'Abbey', 'Muhammad', 'standards', 'Cromwell', 'Be', 'since', 'Lagoon', 'hydroelectric', 'walls', 'May', 'aircraft', 'Get', 'liners', 'rounded', 'port', 'fastest-growing', 'capita', 'feminist', 'sti

In [ ]:
print(model['pail'].shape)
print(np.random.randn(300).shape)

(300,)
(300,)


## Get the vector corresponding to the word using the pretrained model

In [ ]:
pretrained = []

for index, key in enumerate(word2index.keys()):
    try:
        pretrained.append(model[key])
    except:
        #print(index, key)
        pretrained.append(np.random.randn(300))
        
pretrained_vectors = np.vstack(pretrained)
#print(pretrained)

## Modeling 


![alttxt](https://cdn.talentsprint.com/aiml/Casestudies_slides/NLP_with_CNN/NLP_with_CNN.png)





The above image is borrowed from this [paper.](http://www.aclweb.org/anthology/D14-1181)

## Define CNN classifier architecture for classification as per the paper 

In [ ]:
class  CNNClassifier(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, output_size, kernel_dim=100, kernel_sizes=(3, 4, 5), dropout=0.5):
        super(CNNClassifier,self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv2d(1, kernel_dim, (K, embedding_dim)) for K in kernel_sizes])

        # kernal_size = (K,D) 
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(len(kernel_sizes) * kernel_dim, output_size)
    
    
    def init_weights(self, pretrained_word_vectors, is_static=False):
        self.embedding.weight = nn.Parameter(torch.from_numpy(pretrained_word_vectors).float())
        if is_static:
            self.embedding.weight.requires_grad = False


    def forward(self, inputs, is_training=False):
        inputs = self.embedding(inputs).unsqueeze(1)
        inputs = [F.relu(conv(inputs)).squeeze(3) for conv in self.convs]
        inputs = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in inputs]

        concated = torch.cat(inputs, 1)

        if is_training:
            concated = self.dropout(concated)
        out = self.fc(concated) 
        return F.log_softmax(out,1)

## Training the model 

##It takes for a while if you use just cpu.

## Set the parameters

In [ ]:
EPOCH = 5
BATCH_SIZE = 50
KERNEL_SIZES = [2,2,2]
KERNEL_DIM = 100
LR = 0.001

## Set up the defined CNN model and  Initialize embedding matrix using pretrained vectors

In [ ]:
model = CNNClassifier(len(word2index), 300, len(target2index), KERNEL_DIM, KERNEL_SIZES)
model.init_weights(pretrained_vectors) # initialize embedding matrix using pretrained vectors

## Switch on the CUDA

In [ ]:
if USE_CUDA:
    model = model.cuda()

## Define loss function and optimizer

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

## Train the data batch wise

In [ ]:
for epoch in range(EPOCH):
    losses = []
    for i,batch in enumerate(getBatch(BATCH_SIZE, train_data)):
        inputs,targets = pad_to_batch(batch)
        
        model.zero_grad()
        preds = model(inputs, True)
        
        loss = loss_function(preds, targets)
        losses.append(loss.data.item())
        loss.backward()
        
        #for param in model.parameters():
        #    param.grad.data.clamp_(-3, 3)
        
        optimizer.step()
        
        if i % 100 == 0:
            print("[%d/%d] mean_loss : %0.2f" %(epoch, EPOCH, np.mean(losses)))
            losses = []

[0/5] mean_loss : 1.84
[1/5] mean_loss : 0.60
[2/5] mean_loss : 0.19
[3/5] mean_loss : 0.05
[4/5] mean_loss : 0.08


## Predict the test data with the trained model and calculate the test accuracy 

In [ ]:
accuracy = 0
for test in test_data:
    pred = model(test[0].cuda()).max(1)[1]
    pred = pred.data.tolist()[0]
    target = test[1].data.tolist()[0][0]
    if pred == target:
        accuracy += 1

print(accuracy/len(test_data) * 100)

98.35164835164835


## Please answer the questions below to complete the experiment:

In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Good, But Not Challenging for me" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "nn" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["","Yes", "No"]

In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "Very Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook  { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id =return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")

Your submission is successful.
Ref Id: 11806
Date of submission:  19 Dec 2020
Time of submission:  14:00:25
View your submissions: https://aiml.iiith.talentsprint.com/notebook_submissions
